# Goal: Perform a hyperparameter sweep over a set of CNN parameters for regression of Bader charges

In [ ]:
import numpy as np
import pandas as pd
import os

# Machine learning:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, \
 MaxPooling1D, Flatten
from keras import regularizers
from keras import optimizers
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

# Plotting
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

import json
import talos as ta

from trixs.machine_learning.util import onehot_reverse

In [ ]:
data = []
for line in open('./Ti_O_XY.json', 'r'):
    data.append(json.loads(line))

In [ ]:
print("Total of %i data points" % len(data))
print("Keys are %a" % data[0].keys())

# Using mu here not mu0
all_feature_lens = [len(data[ii]['mu']) == len(data[0]['mu'])
                    for ii in range(len(data))]
print("All features same length: %s" % np.all(np.array(all_feature_lens)))


## Classification Problem

In [ ]:
# Only take 4, 5 and 6 coordinated:
to_ignore = [ii for ii in range(len(data)) if data[ii]['coordination'] not in [4, 5, 6]]
features = np.array([data[ii]['mu'] for ii in range(len(data)) if ii not in to_ignore])

# Minus 4 so that 6 - 4 = 2, 5 - 4 = 1 and 4 - 4 = 0; ensures that my one-hot
# decoder/encoder system can work properly!
_targets = np.array([data[ii]['coordination'] - 4 for ii in range(len(data)) if ii not in to_ignore])
targets = onehot_reverse(_targets)
print(features.shape, targets.shape)

In [ ]:
# Generate train/validate/test splits
RANDSTATE=555
x_train, x_test, y_train, y_test = \
  train_test_split(features, targets, test_size=0.1,
                   random_state=RANDSTATE)
x_train, x_valid, y_train, y_valid = \
  train_test_split(x_train, y_train, test_size=0.1,
                   random_state=RANDSTATE)
print("training    %s ~ %s"
      % ((x_train.shape), (y_train.shape)))
print("validation  %s ~ %s"
      % ((x_valid.shape), (y_valid.shape)))
print("testing     %s ~ %s"
      % ((x_test.shape), (y_test.shape)))

# Want to normalize the targets first
y_train = y_train/np.max(y_train, axis=1, keepdims=True)
y_valid = y_valid/np.max(y_valid, axis=1, keepdims=True)
y_test = y_test/np.max(y_test, axis=1, keepdims=True)

In [ ]:
# The values of the input features vary wildly between different
# feature types, over different orders of magnitude. Thus, we
# must scale each feature to zero mean and unit variance to
# avoid problematic issues during training. We fit only on the
# training data, and execute that same scaling on the validation
# and testing data to ensure we do not accidentally bias the
# latter two data sets. Sanity check: printing out the feature/
# target means before and after (averaged over all features/
# targets just to get an idea).

train_feature_mean = np.mean(np.mean(x_train, axis=0))
train_feature_std = np.mean(np.std(x_train, axis=0))

valid_feature_mean = np.mean(np.mean(x_valid, axis=0))
valid_feature_std = np.mean(np.std(x_valid, axis=0))

test_feature_mean = np.mean(np.mean(x_test, axis=0))
test_feature_std = np.mean(np.std(x_test, axis=0))


print("~~~~~~~~~~~~ Mean +/- Stdev before ~~~~~~~~~~~~~~")
print("-------------------------------------------------")
print("Train features %.03f +/- %.03f"
      % (train_feature_mean, train_feature_std))
print("Valid features %.03f +/- %.03f"
      % (valid_feature_mean, valid_feature_std))
print("Test features  %.03f +/- %.03f"
      % (test_feature_mean, test_feature_std))


# Generate a feature and target scaler
feature_scaler = StandardScaler().fit(x_train)

# Utilize that scaler on the datasets

x_train = feature_scaler.transform(x_train)
x_valid = feature_scaler.transform(x_valid)
x_test = feature_scaler.transform(x_test)

train_feature_mean = np.mean(np.mean(x_train, axis=0))
train_feature_std = np.mean(np.std(x_train, axis=0))

valid_feature_mean = np.mean(np.mean(x_valid, axis=0))
valid_feature_std = np.mean(np.std(x_valid, axis=0))

test_feature_mean = np.mean(np.mean(x_test, axis=0))
test_feature_std = np.mean(np.std(x_test, axis=0))

print("~~~~~~~~~~~~~ Mean +/- Stdev after ~~~~~~~~~~~~~~")
print("-------------------------------------------------")
print("Train features %.03f +/- %.03f"
      % (train_feature_mean, train_feature_std))
print("Valid features %.03f +/- %.03f"
      % (valid_feature_mean, valid_feature_std))
print("Test features  %.03f +/- %.03f"
      % (test_feature_mean, test_feature_std))
""";

In [ ]:
def classification_model(x_train, y_train, x_val, y_val, params):

    # Params is the vessel that will carry the optimzation parameters
    dropout = params['dropout']
    act = params['activation_function']
    optimizer = params['optimizer']

    #layers = params['layers']

    model = Sequential()

    # CNN vs. MLP.
    if params['cnn']:
        x_train = np.expand_dims(x_train, axis=-1)
        x_val = np.expand_dims(x_val, axis=-1)
        model.add(Conv1D(params['kernel'],
                         params['n_filters'],
                         strides=params['strides'], padding='valid',
                         activation=act,
                         input_shape=(x_train.shape[1], 1)))
        model.add(MaxPooling1D(pool_size=params['pool_size'],
                               strides=None, padding='valid'))
        model.add(Flatten())
        model.add(Dense(params['layer0'], activation=act))
    else:
        model.add(Dense(params['layer0'], activation=act,
                        input_shape=(x_train.shape[1],)))
        model.add(Dropout(dropout))


    # note the change here relative to the regresion problem
    model.add(Dense(params['layer1'], activation=act))
    model.add(Dropout(dropout))

    model.add(Dense(params['layer2'], activation=act))
    model.add(Dropout(dropout))

    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(loss=params['loss_function'], optimizer=optimizer,
                metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                      batch_size=params['batch_size'],
                      epochs=params['epochs'],
                      validation_data=[x_val, y_val],
                      verbose=0,
                      shuffle=True)

    return history, model

In [ ]:
# Steve, play around with this a bit! Also, I would be careful: the accuracy
# is not a great metric here if you have major class imbalances! :)
params = {
    'layer0': [90,60,45, 30],
    'layer1':[45,60,75], 
    'layer2':[15,30,45],
    'dropout': [0.1],
    'activation_function': ['relu'],
    'optimizer': ['adam'],
    'cnn': [True],
    'kernel': [8,12,14],
    'n_filters': [10,12],
    'strides': [1,2],
    'pool_size': [1,2,3],
    'loss_function': ['categorical_crossentropy'],
    'batch_size': [32],
    'epochs': [30]
}

In [ ]:
t = ta.Scan(x_train, y_train, x_val=x_valid, y_val=y_valid,
            model=classification_model, params=params)

In [ ]:
from IPython.display import display, HTML
pd.set_option('display.max_columns', 30)
pd.set_option('display.width',500)
print(t.data.sort_values(by='val_loss', ascending=True))
display(HTML(t.data.sort_values(by='val_loss', ascending=True).to_html()))

In [ ]:
use_params = {
    'layer0': 90,
    'layer1':60, 
    'layer2':45,
    'dropout': 0.1,
    'activation_function': 'relu',
    'optimizer': 'adam',
    'cnn': True,
    'kernel': 8,
    'n_filters': 10,
    'strides': 1,
    'pool_size': 2,
    'loss_function': 'categorical_crossentropy',
    'batch_size': 32,
    'epochs':25}

In [ ]:
def get_layer_output_gradient_simple_target(model,X,target_output):
    inputs = model.input
    outputs = model.output
    focus = model.output[0][target_output]
    grad = K.gradients(focus, inputs)[0]
    f = K.function([inputs], [outputs,grad])    
    return f([X])

In [ ]:
_, the_model =  classification_model(x_train, y_train, x_valid, y_valid, use_params)

In [ ]:
print(y_valid)

In [ ]:
from tqdm import tqdm_notebook
one_hot_to_coord = {(1,0,0):0, (0,1,0):1, (0,0,1):2 }
validation_by_coords = {(1,0,0):[], (0,1,0):[], (0,0,1):[] }
grads_by_coords = {(1,0,0):[], (0,1,0):[], (0,0,1):[] }
for i in tqdm_notebook(range(len(y_valid))):
    cur_tup =  tuple([int(x) for x in y_valid[i]])
    validation_by_coords[cur_tup].append(x_valid[i])
    exs = x_valid[i].reshape(1,100,1)
    grads_by_coords[cur_tup].append(get_layer_output_gradient_simple_target(the_model,
                                                                            exs,
                                                                            one_hot_to_coord[cur_tup])



In [ ]:
exs = x_valid.reshape(565,100,1)
results, gradients = get_layer_output_gradient_simple_target(the_model, exs, 0)


In [ ]:
class_results = np.argmax(results,axis=1)
#print(class_results)
print(class_results.shape)
print(gradients[0])

In [ ]:
import matplotlib as mpl
X = np.linspace(0,1,100)
#for _ in [0,1]:
#for x in x_valid.reshape(565,100,1):   
cmap = mpl.cm.get_cmap('bwr')
norm = mpl.colors.Normalize(vmin=min(gradients[0]), vmax=max(gradients[0]))
grad_colors = norm(gradients[0].reshape(100))
mu = exs[0].reshape(100)
colors = cmap(grad_colors)
plt.plot(X,mu,zorder=-1)
plt.colorbar()
for i,x in enumerate(X):
    plt.scatter(x,mu[i],color=colors[i])
    




In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
from scipy.interpolate import interp1d
x = np.linspace(0,1,100)
for y in grads_by_coords[(1,0,0)]:
    pass
    #print(x,y)
    #plt.plot(x,y.reshape(100),color='red')
    
megagrad = np.mean([y.reshape(100) for y in grads_by_coords[(1,0,0)]],axis=0)
plt.plot(x,validation_by_coords[(0,1,0)][])
plt.plot(x,megagrad)
print(megagrad.shape)
print(np.sum(megagrad,axis=0).shape)
x_dens = np.linspace(0,1,300)
print(x_dens.shape)
print(x.shape)
#megagrad_sum = np.sum(megagrad,axis=0)
#f = interp1d(x,megagrad_sum)
y_dens = f(x_dens)
#plt.plot(x_dens,y_dens)
#plt.plot(x_dens,gaussian_filter1d(y_dens,sigma=.05))
#plt.plot(x,megagrad,lw=2)
#for y in grads_by_coords[(0,1,0)]:
    #print(x,y)
#    plt.plot(x,y.reshape(100),color='blue')
    
#for y in grads_by_coords[(0,0,1)]:
    #print(x,y)
#    plt.plot(x,y.reshape(100),color='green')
    